# New notebook

In [41]:
import datetime

import pandas as pd
import sqlite3 as db
import yfinance as yf

from sqlalchemy import create_engine
file_storage = "test2"


In [ ]:
symbols_data = [
        {
            "Symbol": "A",
            "Name": "Agilent Technologies Inc",
            "ListedDt": datetime.datetime(2005, 1, 3).isoformat(),
            "LastDt": datetime.datetime(2022, 9, 6).isoformat(),
            "Status": "Active",
        },
        {
            "Symbol": "AA",
            "Name": "Alcoa Corporation",
            "ListedDt": datetime.datetime(2016, 10, 18).isoformat(),
            "LastDt": datetime.datetime(2022, 9, 6).isoformat(),
            "Status": "Active",
        },
        {
            "Symbol": "ZGNX",
            "Name": "Zogenix",
            "ListedDt": datetime.datetime(2010, 11, 23).isoformat(),
            "LastDt": datetime.datetime(2022, 3, 4).isoformat(),
            "Status": "Active",
        }
    ]

update_data = [
        {
            "Symbol": "AA",
            "Name": "Updated Alcoa Corporation",
            "ListedDt": datetime.datetime(2022, 10, 18).isoformat(),
            "LastDt": datetime.datetime(2022, 9, 6).isoformat(),
            "Status": "Active",
        },
        {
            "Symbol": "ZGNX",
            "Name": "Updated Zogenix",
            "ListedDt": datetime.datetime(2012, 11, 23).isoformat(),
            "LastDt": datetime.datetime(2022, 3, 4).isoformat(),
            "Status": "Active",
        },
        {
            "Symbol": "BC",
            "Name": "Basic Company",
            "ListedDt": datetime.datetime(2021, 1, 1).isoformat(),
            "LastDt": datetime.datetime(2022, 3, 4).isoformat(),
            "Status": "Active",
        }
    ]

In [ ]:
source = pd.DataFrame(symbols_data)
new_data = pd.DataFrame(update_data)

In [ ]:
suffixe_new = '_new'
cols_old = source.columns
cols_new = [f"{col}{suffixe_new}" for col in cols_old]
cols_new[0] = cols_old[0]

In [ ]:
cols_old

In [ ]:
cols_new

In [ ]:
merged = pd.merge(source, new_data, on='Symbol', how='outer', indicator=True, suffixes=['', '_new'])
merged

In [ ]:
old = merged.loc[merged._merge == 'left_only'][cols_old]
old

In [ ]:
updates = merged.loc[merged._merge == 'both'][cols_new]
updates.columns = cols_old
updates

In [ ]:
new_sym = merged.loc[merged._merge == 'right_only'][cols_new]
new_sym.columns = cols_old
new_sym

In [ ]:
new_save = pd.concat([old, updates, new_sym], ignore_index=True)
new_save

In [ ]:
df1 = pd.DataFrame(symbols_data)
df2 = pd.DataFrame(update_data)
set1 = set()
set1.update(df1.Symbol)
set1.update(df2.Symbol)
set1


# YAHOO SECTOR INFO

In [ ]:
import datetime

import pandas as pd
import yfinance as yf

In [ ]:
t1 = yf.Ticker('ZGNX')
t1.info

In [ ]:
t1 = yf.Ticker('BBDD.L')
print(t1.info.get('exchange'))
print(t1.info.get('longName'))
t1.info.keys()

In [ ]:
def get_sector_info(ticker):
    t1 = yf.Ticker(ticker)

    info_dict = {
    'Symbol': [ticker], 
    'Sector': [t1.info.get('sector')], 
    'Indutry': [t1.info.get('industry')],
    'QuoteType': [t1.info.get('quoteType')]
    }

    return pd.DataFrame(info_dict)

    
    

In [ ]:
df = get_sector_info('TSLA')
df

In [ ]:
symbols = pd.read_csv(f'data/nasdaq.csv', index_col=False)

In [ ]:
yahoo_codes = [
    {'Code': 'NASDAQ', 'Name':'NASDAQ Stock Exchange', 'Country':'USA', 'Suffix':''},
    {'Code': 'AMEX', 'Name':'American Stock Exchange', 'Country':'USA', 'Suffix':''},
    {'Code': 'ASX', 'Name':'Australian Stock Exchange', 'Country':'Australia', 'Suffix':'.AX'},
    {'Code': 'LSE', 'Name':'London Stock Exchange', 'Country':'United Kingdom', 'Suffix':'.L'},
    {'Code': 'NYSE', 'Name':'New York Stock Exchange', 'Country':'USA', 'Suffix':''},
    {'Code': 'SGX', 'Name':'Singapore Stock Exchange', 'Country':'Republic of Singapore', 'Suffix':'.SI'},
    {'Code': 'TSX', 'Name':'Toronto Stock Exchange', 'Country':'Canada', 'Suffix':'.TO'},
    {'Code': 'TSXV', 'Name':'Toronto Venture Exchange', 'Country':'Canada', 'Suffix':'.V'},
]

In [ ]:
from backtest_lib.symbols import OnlineSymbolsSource, EndOfDayData, FirstRateData


In [ ]:
df = EndOfDayData(exchange='NASDAQ')
df.load_from_csv('data','nasdaq.csv')
df.augment_symbols_data()
#print(df.data)

In [36]:
import pandas as pd
import yfinance as yf


In [6]:
yahoo_ticker = 'AAB.TO'
exchange = 'TSX'
yahoo = yf.Ticker(yahoo_ticker)

In [55]:
base_info = {
    'Symbol':'A',
    'Name':'Agilent Technologies Inc',
    'ListedDt':'2005-01-03T00:00:00',
    'LastDt':'2022-09-22T00:00:00',
    'Status':'Active'
}
base_serie = pd.Series(base_info)
sym_info = {
    'Sector': yahoo.info.get('sector'),
    'Industry': yahoo.info.get('industry'),
    'Type': yahoo.info.get('quoteType'),
    'Source': yahoo.info.get('quoteTypeSourceName'),
    'Website': yahoo.info.get('website'),
    'LogoUrl': yahoo.info.get('logo_url'),
    'Exchange': yahoo.info.get('exchange'),
    'ShortName': yahoo.info.get('shortName'),
    'LongName': yahoo.info.get('longName'),
    'Market': yahoo.info.get('market'),
    'FundFamily': yahoo.info.get('fundFamily'),
    'MarketCap': yahoo.info.get('marketCap'),
    'YahooTicker': yahoo_ticker,
    'ExchangeCode': exchange
}
sym_serie = pd.Series(sym_info)


In [57]:
print(f"{type(base_info)}, {type(sym_info)}")

<class 'dict'>, <class 'dict'>


In [56]:
print(f"{type(base_serie)}, {type(sym_serie)}")

<class 'pandas.core.series.Series'>, <class 'pandas.core.series.Series'>


In [ ]:
pd.concat([base_serie, sym_serie])


In [61]:
augmented = base_info | sym_info
augmented

{'Symbol': 'A',
 'Name': 'Agilent Technologies Inc',
 'ListedDt': '2005-01-03T00:00:00',
 'LastDt': '2022-09-22T00:00:00',
 'Status': 'Active',
 'Sector': 'Financial Services',
 'Industry': 'Asset Management',
 'Type': 'EQUITY',
 'Source': None,
 'Website': 'https://www.aberdeen.green',
 'LogoUrl': 'https://logo.clearbit.com/aberdeen.green',
 'Exchange': 'TOR',
 'ShortName': 'ABERDEEN INTERNATIONAL INC.',
 'LongName': 'Aberdeen International Inc.',
 'Market': 'ca_market',
 'FundFamily': None,
 'MarketCap': 10141389,
 'YahooTicker': 'AAB.TO',
 'ExchangeCode': 'TSX'}

In [63]:
df = pd.DataFrame([augmented])
df

,Symbol,Name,ListedDt,LastDt,Status,Sector,Industry,Type,Source,Website,LogoUrl,Exchange,ShortName,LongName,Market,FundFamily,MarketCap,YahooTicker,ExchangeCode
0,A,Agilent Technologies Inc,2005-01-03T00:00:00,2022-09-22T00:00:00,Active,Financial Services,Asset Management,EQUITY,None,https://www.aberdeen.green,https://logo.clearbit.com/aberdeen.green,TOR,ABERDEEN INTERNATIONAL INC.,Aberdeen International Inc.,ca_market,None,10141389,AAB.TO,TSX


In [71]:
df.to_csv('augmented.csv', header=False, index=False, mode='a')

In [ ]:
df

In [42]:
github = 'https://raw.githubusercontent.com/MapleFrogStudio/DATASETS/main/STOCK_SYMBOLS/YAHOO'
amex = pd.read_csv(f'{github}/amex.csv', index_col='Symbol')
asx = pd.read_csv(f'{github}/asx.csv', index_col='Symbol')
lse = pd.read_csv(f'{github}/lse.csv', index_col='Symbol')
nasdaq = pd.read_csv(f'{github}/nasdaq.csv', index_col='Symbol')
nyse = pd.read_csv(f'{github}/nyse.csv', index_col='Symbol')
tsx = pd.read_csv(f'{github}/tsx.csv', index_col='Symbol')
tsxv = pd.read_csv(f'{github}/tsxv.csv')

In [23]:
#git2 = 'https://raw.githubusercontent.com/MapleFrogStudio/DATASETS/main/STOCK_SYMBOLS/CSV/eoddata_tsxv.csv'
#tsxv = pd.read_csv(f'{git2}')

In [43]:
print(f"amex   : {len(amex)}")
print(f"asx    : {len(asx)}")
print(f"lse    : {len(lse)}")
print(f"nasdaq : {len(nasdaq)}")
print(f"nyse   : {len(nyse)}")
print(f"tsx    : {len(tsx)}")
print(f"tsxv   : {len(tsxv)}")
print(f"Total  : {len(amex)+len(asx)+len(lse)+len(nasdaq)+len(nyse)+len(tsx)+len(tsxv)}")

amex   : 2852
asx    : 3763
lse    : 5937
nasdaq : 5713
nyse   : 3640
tsx    : 2473
tsxv   : 1686
Total  : 26064


In [40]:
tsxv.describe(include='all')

,Symbol,Name
count,1679,1679
unique,1679,1669
top,A,Lifeist Wellness Inc
freq,1,3
